In [1]:
import tensorflow as tf
import numpy as np

config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

In [2]:
# 辅助方法

# 卷积
def conv(x, filter_height, filter_width, filters_num, stride_x, stride_y, name, padding='SAME', groups=1): # groups: 分成多个部分
    input_channels = int(x.get_shape()[-1]) # 输入通道数
    convolve = lambda i, k: tf.nn.conv2d(i, k, strides=[1, stride_x, stride_y, 1], padding=padding) # 卷积
    
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable('weights', shape=[filter_height, filter_width, input_channels/groups, filters_num])
        bias = tf.get_variable('bias', shape=[filters_num])
        
        if groups == 1:
            conv = convolve(x, weights)
        else:
            input_groups = tf.split(value=x, num_or_size_splits=groups, axis=3) # 切分
            weight_groups = tf.split(value=weights, num_or_size_splits=groups, axis=3)
            output_groups = [convolve(i, k) for i, k in zip(input_groups, weight_groups)] # 分别卷积
            conv = tf.concat(values=output_groups, axis=3) # 拼接
        
        z = tf.reshape(tf.nn.bias_add(conv, bias), conv.get_shape().as_list())
        relu = tf.nn.relu(z, name=scope.name)
        return relu  

# 最大池化
def max_pool(x, filter_height, filter_width, stride_x, stride_y, name, padding='SAME'):
    return tf.nn.max_pool(x, [1, filter_height, filter_width, 1], strides=[1, stride_x, stride_y, 1], padding=padding, name=name)

# 局部响应归一化
def lrn(x, radius, alpha, beta, name, bias=1.0):
    return tf.nn.lrn(x, depth_radius=radius, alpha=alpha, beta=beta, bias=bias, name=name) # bias对应k, radius对应n/2

# 全连接
def fc(x, num_in, num_out, name, relu=True):
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable('weights', shape=[num_in, num_out])
        bias = tf.get_variable('bias', shape=[num_out])
        z = tf.nn.xw_plus_b(x, weights, bias, name=scope.name)
        
        if relu == True:
            act = tf.nn.relu(z)
        else:
            act = z
    return act

# Dropout
def dropout(x, keep_prob):
    return tf.nn.dropout(x, keep_prob)

In [3]:
# AlexNet模型

class AlexNetModel(object):
    
    def __init__(self, num_classes=1000, keep_prob=0.5, skip_layer=[], weights_path='DEFAULT'):
        self.num_classes = num_classes
        self.keep_prob = keep_prob
        self.skip_layer = skip_layer
        
        if weights_path == 'DEFAULT':
            self.weights_path = 'bvlc_alexnet.npy'
        else:
            self.weights_path = weights_path
    
    def inference(self, x, training=False): # 模型
        # conv1: CONV --> POOL --> LRN
        conv1 = conv(x, 11, 11, 96, 4, 4, padding='VALID', name='conv1')
        pool1 = max_pool(conv1, 3, 3, 2, 2, padding='VALID', name='pool1')
        norm1 = lrn(pool1, 2, 2e-05, 0.75, name='norm1')
        
        # conv2: CONV --> POOL --> LRN with 2 Groups
        conv2 = conv(norm1, 5, 5, 256, 1, 1, groups=2, name='conv2')
        pool2 = max_pool(conv2, 3, 3, 2, 2, padding='VALID', name='pool2')
        norm2 = lrn(pool2, 2, 2e-05, 0.75, name='norm2')
        
        # conv3: CONV 
        conv3 = conv(norm2, 3, 3, 384, 1, 1, name='conv3')
        
        # conv4: CONV with 2 Groups
        conv4 = conv(conv3, 3, 3, 384, 1, 1, groups=2, name='conv4')
        
        # conv5: CONV --> PooL with 2 Groups
        conv5 = conv(conv4, 3, 3, 256, 1, 1, groups=2, name='conv5')
        pool5 = max_pool(conv5, 3, 3, 2, 2, padding='VALID', name='pool5')
        
        # fc6: Flatten --> FC --> Dropout
        flattened  = tf.reshape(pool5, [-1, 6*6*256])
        fc6 = fc(flattened, 6*6*256, 4096, name='fc6')
        if training:
            fc6 = dropout(fc6, self.keep_prob)
        
        #fc7: FC --> Dropout
        fc7 = fc(fc6, 4096, 4096, name='fc7')
        if training:
            fc7 = dropout(fc7, self.keep_prob)
            
        #fc8: FC
        self.score = fc(fc7, 4096, self.num_classes, relu=False, name='fc8')
        
        return self.score
    
    def loss(self, batch_x, batch_y): # 损失
        y_predict = self.inference(batch_x, training=True)
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict, labels=batch_y))
        return self.loss
    
    def optimize(self, learning_rate, train_layers=[]): # 优化
        var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers] # 获取可训练的所有参数
        return tf.train.AdamOptimizer(learing_rate).minimize(self.loss, var_list=var_list)
    
    def load_original_weights(self, session): # 导入训练好的权重
        weights_dict = np.load(self.weights_path, encoding='bytes').item()
        
        for op_name in weights_dict:
            if op_name not in self.skip_layer:     
                with tf.variable_scope(op_name, reuse=True):
                    for data in weights_dict[op_name]:
                        if len(data.shape) == 1:
                            var = tf.get_variable('bias')
                            session.run(var.assign(data))
                        else:
                            var = tf.get_variable('weights')
                            session.run(var.assign(data))  

In [4]:
# 保存为pb模型

inputs = tf.placeholder(tf.float32, [1, 227, 227, 3])

model = AlexNetModel()
score = model.inference(inputs)
softmax = tf.nn.softmax(score, name='prob')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.load_original_weights(sess)
    
    constant_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, ['prob'])
    
    with tf.gfile.GFile('./model/frozen_alexnet_model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())

INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.


In [ ]:
# 测试模型

import matplotlib.pyplot as plt
import cv2
import os

img_dir = os.path.join(os.getcwd(), 'images')
img_file = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.jpeg')]

imgs = []
for f in img_file:
    imgs.append(cv2.imread(f))

fig = plt.figure(figsize=(15, 6))
for i, img in enumerate(imgs):
    fig.add_subplot(1, 3, i+1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

In [ ]:
from caffe_classes import class_names # 分类类别名

imagenet_mean = np.array([104., 117., 124.], dtype=np.float32) # ImageNet中的图片像素均值
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
model = AlexNetModel()
score = model.inference(x)

softmax = tf.nn.softmax(score)

In [ ]:
writer = tf.summary.FileWriter('./graph/alexnet', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.load_original_weights(sess)
    
    fig2 = plt.figure(figsize=(15, 6))
    
    for i, image in enumerate(imgs):
        img = cv2.resize(image.astype(np.float32), (227, 227))
        img -= imagenet_mean
        img = img.reshape((1, 227, 227, 3))
        
        probs = sess.run(softmax, feed_dict={x: img})
        class_name = class_names[np.argmax(probs)]
        
        writer.close()
        fig2.add_subplot(1,3,i+1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title("Class: " + class_name + ", probability: %.4f" %probs[0,np.argmax(probs)])
        plt.axis('off')